<a href="https://colab.research.google.com/github/TheUnnoticable/Projects/blob/main/Linear_Regression_FCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

code = '''
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
'''

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# so the way I'm going to convert categorical data to numerical data is through the "get_dummies" method
CATEGORICAL_FEATURES = ["sex", "smoker", "region"]
dataset = pd.get_dummies(dataset, columns=CATEGORICAL_FEATURES, prefix='', prefix_sep='')
dataset.head()

In [ ]:
# ok I guess I will group similar customers into clusters using KMeans!
from sklearn.cluster import KMeans
features=['age', 'bmi', 'no', 'yes', 'children', 'northeast', 'northwest', 'southeast', 'southwest']
kmeans = KMeans(n_clusters=2)
kmeans.fit(dataset[features])

dataset['cust_type'] = kmeans.predict(dataset[features])

# does this mean I have to pop out the rest?
# yes.... yes it does

In [ ]:
dataset['more_than_1_child'] = dataset.children.apply(lambda x:1 if x>1 else 0)
dataset['location_north']=dataset.apply(lambda x: (x['northeast'] or x['northwest']), axis=1)
dataset['location_south']=dataset.apply(lambda x: (x['southeast'] or x['southwest']), axis=1)
#NW, NE, SW, SE = dataset.pop("northwest"), dataset.pop("northeast"), dataset.pop("southwest"), dataset.pop("southeast")

In [ ]:
children = dataset.pop("children")
dataset.head()

In [ ]:
import seaborn as sns
sns.heatmap(dataset[['age', 'bmi', 'children', 'expenses']].corr(), cmap='Blues', annot=True)
#sns.pairplot(data=dataset, diag_kind='kde')

In [ ]:
# NOW.... we can split the data into training and test sets!
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
# ^ another way is... test_dataset = dataset[~dataset.index.isin(train_dataset.index)]

In [ ]:
train_labels = train_dataset.pop("expenses")
test_labels = test_dataset.pop("expenses")

In [ ]:
# look how much the data varies bro
train_dataset.describe().transpose()

In [ ]:
# So... let's normalize it!
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))
print(normalizer.mean.numpy())

In [ ]:
# Create and train model!!
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=16, activation="relu"),
    layers.Dense(units=4, activation="relu"),
    layers.Dense(units=1)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.7),
    loss='mean_absolute_error'
)

history = model.fit(
    train_dataset,
    train_labels,
    epochs=900,
    verbose=0,
    validation_split=0.2)

results = model.evaluate(test_dataset, test_labels, verbose=0)

In [ ]:
results
# (1) 13864.5234375
# (2) 12540.5205078125
# (3) 10890.16015625
# (4) 9226.2548828125
# (5) 7525.42431640625
# (6) 4436.88427734375
# (7) 4068.13671875
# (8) 4078.349853515625
# (9) 2549.600341796875


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
mae = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
